# Wigner Transform

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pmd_beamphysics.wavefront.wavefront import Wavefront
from pmd_beamphysics.wavefront.gaussian import add_gaussian
from pmd_beamphysics.wavefront.propagators import drift_wavefront
from pmd_beamphysics.wavefront.wigner import wigner

from pmd_beamphysics.plot import plot_2d_density_with_marginals

from ipywidgets import interact

from scipy.constants import epsilon_0

import numpy as np

import os

import matplotlib.pyplot as plt

In [ ]:
W = Wavefront(
    Ex=np.zeros((201, 201, 51)),
    dx=10e-6,
    dy=10e-6,
    dz=10e-6,
    wavelength=1e-9,
)
w0 = 100e-6
# zR = np.pi * w0**2 / W0.wavelength

add_gaussian(W, z=0, w0=w0, energy=1.2345)
# add_gaussian(W, z=0, w0=w0, energy=1.2345, x0 = -.0001)
# add_gaussian(W, z=0, w0=w0, energy=1.2345, x0 =  .0001, phase =0*np.pi)


W.plot()

In [ ]:
W1 = drift_wavefront(W, 100)
# W1 = drift_wavefront_advanced(W, 100, Rcurv=10)
W1.plot()
W1.to_kspace().plot()

In [ ]:
W.to_kspace().sigma_kx

In [ ]:
W1.to_kspace().sigma_kx

In [ ]:
Ex0 = W.Ex[:, W.ny // 2, W.nz // 2]  # slice along y=0, z=0

In [ ]:
# Energy in this slice
np.sum(np.abs(Ex0) ** 2) * W.dx * W.dy * W.dz * epsilon_0 / 2

In [ ]:
w, tvec, fvec = wigner(Ex0, dx=W.dx)  # Units are (V/m)^2 * m = V^2 / m

dt = np.diff(tvec)[0]
df = np.diff(fvec)[0]

In [ ]:
# The sum gives the same energy.
# Because the  (V/m)^2 / (1/m) = V^2 / m
np.sum(w) * W.dx * W.dy * W.dz * epsilon_0 / 2

In [ ]:
plot_2d_density_with_marginals(
    w * W.dx * W.dy * W.dz * epsilon_0 / 2,
    dx=dt,
    dy=df,
    x_name=r"$x$",
    x_units="m",
    y_name=r"$f$",
    y_units="1/m",
    z_name="energy density",
    z_units="J",
    cmap="bwr",
    vcenter=0,
)

# Make movie

In [ ]:
def plot1(z):
    W1 = drift_wavefront(W, z)
    # W1 = drift_wavefront_advanced(W, z, Rcurv=100)
    # print(W1.to_kspace().sigma_kx)
    Ex0 = W1.Ex[:, W1.ny // 2, W1.nz // 2]
    w, tvec, fvec = wigner(Ex0, dx=W1.dx)
    dt = np.diff(tvec)[0]
    df = np.diff(fvec)[0]
    plot_2d_density_with_marginals(
        w * W1.dx * W1.dy * W1.dz * epsilon_0 / 2,
        dx=dt,
        dy=df,
        x_name=r"$x$",
        x_units="m",
        y_name=r"$f$",
        y_units="1/m",
        z_name="energy density",
        z_units="J",
        cmap="bwr",
        # vcenter=0,
        #   xlim=(-.001, .001),
        #    ylim=(-10000, 10000),
    )


plot1(100)

In [ ]:
interact(plot1, z=(1, 100, 1))

In [ ]:
# Step 1: Create a directory to store frames
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Step 2: Generate frames
z_values = np.linspace(0, 100, 100)

for i, z in enumerate(z_values):
    plot1(z)  # Call your plot1(z) function
    fig = plt.gcf()
    frame_path = os.path.join(output_dir, f"frame_{i:04d}.png")  # Zero-padded filenames
    plt.savefig(frame_path)  # Save the current frame
    plt.close(fig)  # Close the figure to free memory

print(f"Frames saved in {output_dir}")

In [ ]:
# Step 3: Combine frames into a video
# ffmpeg command: Adjust the framerate (-r) as needed
os.system(
    f"ffmpeg -r 30 -i {output_dir}/frame_%04d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p output.mp4"
)
print("Video created: output.mp4")